In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import pandas as pd
import io

In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame.

In [3]:
URL = 'https://realpython.github.io/fake-jobs/'

response = requests.get(URL)

In [4]:
response.status_code

200

#### 1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  


In [6]:
soup = BeautifulSoup(response.text)

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Fake Python
  </title>
  <link href="https://cdn.jsdelivr.net/npm/bulma@0.9.2/css/bulma.min.css" rel="stylesheet"/>
 </head>
 <body>
  <section class="section">
   <div class="container mb-5">
    <h1 class="title is-1">
     Fake Python
    </h1>
    <p class="subtitle is-3">
     Fake Jobs for Your Web Scraping Journey
    </p>
   </div>
   <div class="container">
    <div class="columns is-multiline" id="ResultsContainer">
     <div class="column is-half">
      <div class="card">
       <div class="card-content">
        <div class="media">
         <div class="media-left">
          <figure class="image is-48x48">
           <img alt="Real Python Logo" src="https://files.realpython.com/media/real-python-logo-thumbnail.7f0db70c2ed2.jpg?__no_cf_polish=1"/>
          </figure>
         </div>
         <div class="media-content">
          <h2 c

a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.

In [9]:
soup.find('h2').text

'Senior Python Developer'

b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.  


In [11]:
job_titles = [job.text.strip() for job in soup.find_all('h2')]

print(f"Amount of job titles: {len(job_titles)}")

Amount of job titles: 100


c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  


In [13]:
company_names = [company.text.strip() for company in soup.find_all('h3')]
location_names = [location.text.strip() for location in soup.find_all(class_='location')]
date_values = [date.text.strip() for date in soup.find_all('time')]

print(f"Amount of company names: {len(company_names)}")
print(f"Amount of location names: {len(location_names)}")
print(f"Amount of date values: {len(date_values)}")

Amount of company names: 100
Amount of location names: 100
Amount of date values: 100


d. Take the lists that you have created and combine them into a pandas DataFrame. 


In [15]:
job_listing = pd.DataFrame({
    'job_title': job_titles,
    'company_name': company_names,
    'location': location_names,
    'date_publish': date_values
})

job_listing.head()

,job_title,company_name,location,date_publish
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08


#### 2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.   

a. First, use the BeautifulSoup find_all method to extract the urls.  


In [18]:
soup.find(class_="card-footer").findAll('a')[1].get('href')

'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'

In [19]:
apply_urls = [url.get('href') for url in soup.find_all('a', string="Apply")]
print(f"Amount of apply urls: {len(apply_urls)}")

Amount of apply urls: 100


b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.


In [21]:
urls = []

for index, row in job_listing.iterrows():
    job_title = row['job_title'].lower()

    replace_list = ['(', ')', ',']

    for char in replace_list:
        job_title = job_title.replace(char, '')

    job_title = job_title.replace('/', ' ')
    
    job_url = 'https://realpython.github.io/fake-jobs/jobs/' + job_title.replace(" ", "-") + '-' + str(index) + '.html'
    urls.append(job_url)

urls[0]

'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'

In [22]:
job_listing['url_scraped'] = apply_urls
job_listing['url_generated'] = urls

job_listing.head()

,job_title,company_name,location,date_publish,url_scraped,url_generated
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/se...,https://realpython.github.io/fake-jobs/jobs/se...
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/en...,https://realpython.github.io/fake-jobs/jobs/en...
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/le...,https://realpython.github.io/fake-jobs/jobs/le...
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fi...,https://realpython.github.io/fake-jobs/jobs/fi...
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/pr...,https://realpython.github.io/fake-jobs/jobs/pr...


#### 3. Finally, we want to get the job description text for each job.  


    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  


In [25]:
URL_desc = 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'

response_desc = requests.get(URL_desc)
soup_desc = BeautifulSoup(response_desc.text)

In [26]:
soup_desc.find(class_="content").find('p').text.strip()

'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.'

b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  


In [28]:
def get_job_description(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        
        description = soup.find(class_="content").find('p').text.strip()
        
        return description
    except:
        print(f"Error for {url}")

c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.

In [30]:
job_listing['job_description'] = job_listing['url_generated'].apply(get_job_description)

job_listing.head()

,job_title,company_name,location,date_publish,url_scraped,url_generated,job_description
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/se...,https://realpython.github.io/fake-jobs/jobs/se...,Professional asset web application environment...
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/en...,https://realpython.github.io/fake-jobs/jobs/en...,Party prevent live. Quickly candidate change a...
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/le...,https://realpython.github.io/fake-jobs/jobs/le...,Administration even relate head color. Staff b...
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fi...,https://realpython.github.io/fake-jobs/jobs/fi...,Tv program actually race tonight themselves tr...
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/pr...,https://realpython.github.io/fake-jobs/jobs/pr...,Traditional page a although for study anyone. ...
